####                     Introduction

 In this project, I've worked with data from the 2017 NBA Season (a pretty exciting one) to perform Data Wrangling, and then some analysis.

I've started by performing some Data Wrangling techniques to join the data from the season with that of players. We'll then perform different modifications and cleaning tasks to make sure our data is ready for analysis.

Finally, we'll perform some analysis using Group By and Transform operations.

In [28]:
import pandas as pd
players_df = pd.read_csv('player_data.csv')
s2017_df = pd.read_csv('2017_season_data.csv')

#### Merging the data
Let's start by merging the data from the season with player's data.

In [29]:
# Merge s2017_df and players_df with a left join.
df = s2017_df.merge(players_df, how = 'left', left_on = 'Player', right_on = 'name')

In [49]:
# checking if there are any missmatches
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 551 entries, 0 to 604
Data columns (total 40 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Player      551 non-null    object        
 1   Pos         551 non-null    object        
 2   Age         551 non-null    float64       
 3   Tm          551 non-null    object        
 4   G           551 non-null    float64       
 5   GS          551 non-null    float64       
 6   MP          551 non-null    float64       
 7   ORB%        551 non-null    float64       
 8   DRB%        551 non-null    float64       
 9   TRB%        551 non-null    float64       
 10  AST%        551 non-null    float64       
 11  STL%        551 non-null    float64       
 12  BLK%        551 non-null    float64       
 13  TOV%        549 non-null    float64       
 14  FG          551 non-null    float64       
 15  FGA         551 non-null    float64       
 16  3P          551 non-null    flo

In [46]:
#check how many missmathces there are under each row.
df.isna().sum().tail()

height          0
weight          0
birth_date      0
college       103
Team            0
dtype: int64

In [32]:
# Extracting the names of players that couldn't be matched.
player_misses=df[df['name'].isna()]
player_misses= player_misses['Player']
player_misses = list(player_misses)

In [33]:
# Modifying players_df with the correct names to re-try a successful merge.
player_names_replacements = {
     "Luc Mbah a Moute": "Luc Mbah",
     "James Michael McAdoo": "James Michael",
     "Sheldon Mac": "Sheldon McClellan",
     "Metta World Peace": "Metta World",
}

for old_name, new_name in player_names_replacements.items():
    players_df.loc[players_df['name'] == old_name, 'name'] = new_name


In [34]:
# Performing the merge between s2017_df and players_df again, this time, without misses.
df = s2017_df.merge(players_df, how = 'left', left_on = 'Player', right_on = 'name')

In [35]:
# Removing unnecessary columns.
df.drop(columns = [
    "Year",
    "PER",
    "TS%",
    "3PAr",
    "FTr",
    "USG%",
    "blanl",
    "OWS",
    "DWS",
    "WS",
    "WS/48",
    "blank2",
    "OBPM",
    "DBPM",
    "BPM",
    "VORP",
    "FG%",
    "3P%",
    "eFG%",
    "FT%",
    "name",
], inplace = True
)

In [36]:
# Rename teams to their full names.

team_mapping = {
    "OKC": "Oklahoma City Thunder",
    "DAL": "Dallas Mavericks",
    "BRK": "Brooklyn Nets",
    "SAC": "Sacramento Kings",
    "NOP": "New Orleans Pelicans",
    "MIN": "Minnesota Timberwolves",
    "SAS": "San Antonio Spurs",
    "IND": "Indiana Pacers",
    "MEM": "Memphis Grizzlies",
    "POR": "Portland Trail Blazers",
    "CLE": "Cleveland Cavaliers",
    "LAC": "Los Angeles Clippers",
    "PHI": "Philadelphia 76ers",
    "HOU": "Houston Rockets",
    "MIL": "Milwaukee Bucks",
    "NYK": "New York Knicks",
    "DEN": "Denver Nuggets",
    "ORL": "Orlando Magic",
    "MIA": "Miami Heat",
    "PHO": "Phoenix Suns",
    "GSW": "Golden State Warriors",
    "CHO": "Charlotte Hornets",
    "DET": "Detroit Pistons",
    "ATL": "Atlanta Hawks",
    "WAS": "Washington Wizards",
    "LAL": "Los Angeles Lakers",
    "UTA": "Utah Jazz",
    "BOS": "Boston Celtics",
    "CHI": "Chicago Bulls",
    "TOR": "Toronto Raptors"
}

df['Team'] = df['Tm'].replace(team_mapping)

In [37]:
# Convert birthday to a datetime object. 

df['birth_date'] = pd.to_datetime(df['birth_date'])

In [38]:
# Drop all Players with team "TOT". This team doesn't exist in reality it's just a placeholder for
# players who have transferred teams in this season.

df.drop(df.loc[df['Tm'] == 'TOT'].index, inplace = True)


## Analysis

1. What's the team with the most players in the league?

In [56]:
# let's list them out grouping them by team and then ordering them by descending order.
df.groupby('Team')['Player'].size().sort_values(ascending = False).head()

Team
New Orleans Pelicans    27
Dallas Mavericks        24
Atlanta Hawks           22
Cleveland Cavaliers     22
Philadelphia 76ers      22
Name: Player, dtype: int64

2. What's the team with the lowest `FG`?

In [55]:
df.groupby('Team')['FG'].sum().sort_values(ascending=True).head()

Team
Dallas Mavericks     2968.0
Memphis Grizzlies    2984.0
Utah Jazz            3033.0
Charlotte Hornets    3093.0
Brooklyn Nets        3102.0
Name: FG, dtype: float64

3. What's the team with the best `FG%`?

In [54]:
fg_perc_per_team = df.groupby('Team')[['FG','FGA']].sum()
fg_perc_per_team['FG%'] = fg_perc_per_team['FG'] / fg_perc_per_team['FGA']
fg_perc_per_team.sort_values(by= 'FG%', ascending= False).head()

,FG,FGA,FG%
Team,,,
Golden State Warriors,3532.0,7140.0,0.494678
San Antonio Spurs,3470.0,7284.0,0.476387
Los Angeles Clippers,3242.0,6819.0,0.475436
Washington Wizards,3388.0,7136.0,0.474776
Milwaukee Bucks,3190.0,6737.0,0.473505


4. What's the difference between the best and worst 3P shooters (by position)?

In [58]:
# First we calculate the accuracy of 3P throwers by position:
top_3p_by_position = df.groupby('Pos')[['3P','3PA']].apply( lambda rows: rows['3P'].sum() / rows['3PA'].sum()).sort_values(ascending = False)

# We can then calculate the difference in accuracy between the max value and the min value:
print(top_3p_by_position.max() - top_3p_by_position.min())

0.024253659969040164


5. Find the best scorers in each team

In [61]:
# First we must compute the best scorer per team, we can do it with the transform function:
df["Best Score per Team"] = df.groupby('Team')['PTS'].transform('max')

# Then, it's just about finding the player that matches that best score:
best_scorers_per_team = df.loc[
    df['PTS'] == df["Best Score per Team"],
    ['Player', 'Team', 'Pos', 'PTS']
].sort_values(by='PTS', ascending=False)

best_scorers_per_team.head()

,Player,Team,Pos,PTS
567,Russell Westbrook,Oklahoma City Thunder,PG,2558.0
214,James Harden,Houston Rockets,PG,2356.0
525,Isaiah Thomas,Boston Celtics,PG,2199.0
122,Anthony Davis,New Orleans Pelicans,C,2099.0
538,Karl-Anthony Towns,Minnesota Timberwolves,C,2061.0


6. Which team has the 'youngest squad', by average player age?

In [65]:
#We can use the birth_date column to compute the mean. Pandas correctly computes the mean of Timestamp series:
df.groupby('Team')['birth_date'].mean().sort_values(ascending=False).head()



Team
Portland Trail Blazers   1992-03-14 08:00:00.000000000
Toronto Raptors          1991-04-16 16:56:28.235294080
Boston Celtics           1991-04-04 19:12:00.000000000
Orlando Magic            1991-01-31 16:25:15.789473664
Denver Nuggets           1991-01-14 17:41:03.157894784
Name: birth_date, dtype: datetime64[ns]